In [1]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import datetime
from datetime import datetime
import configparser
import os

from time import sleep
from pathlib import Path
import numpy as np

ee.Authenticate()
ee.Initialize()

In [2]:
config = configparser.ConfigParser()
config.read("config.ini")

# GEE

cfg_nuts = config["gee"]["NUTS"]
cfg_NDVI_LS_mean_ts_ASSET = config["gee"]["NDVI_LS_mean_ts_ASSET"]  # Landsat full timeseries
cfg_gee_assets = config["gee"]["gee_assets"]
cfg_FUA = config["gee"]["FUA"]
cfg_GDRIVE_BASE_URL = config["gee"]["GDRIVE_BASE_URL"]
cfg_GEE_PERCENTILES_FILE_ID = config["gee"]["GEE_PERCENTILES_FILE_ID"]
cfg_GDRIVE_EXPORT_DIRECTORY = config["gee"]["GDRIVE_EXPORT_DIRECTORY"] # Directory in GEE to export results
cfg_WorldPop = config["gee"]["WorldPop"]


# Dates
cfg_ls5_startDate =config["dates"]["ls5_startDate"]
cfg_ls8_endDate = config["dates"]["ls8_endDate"]


# Settings

cfg_OUTPUT_DIR = config["settings"]["OUTPUT_DIR"]  # Directory in local machine
cfg_NUTS_FILTER_COLUMN = config["settings"]["NUTS_FILTER_COLUMN"]
cfg_NUTS_CNTR_CODE = config["settings"]["NUTS_CNTR_CODE"]
cfg_FUA_FILTER_COLUMN = config["settings"]["FUA_FILTER_COLUMN"]
cfg_FCOVER_mean_FUA_CSV= config["settings"]["FCOVER_mean_FUA_CSV"]

NUTS_CNTR_CODE = cfg_NUTS_CNTR_CODE


OUTPUT_DIR = Path(os.getcwd()).parent / cfg_OUTPUT_DIR



# Load NDVI timeseries (as multiband image)

In [6]:
# read NDVI asset (its a multiband image with bands named as NDVI_{year})
NDVI_LS = ee.Image(f"{cfg_gee_assets}{cfg_NDVI_LS_mean_ts_ASSET}") 

# get band names
bands = NDVI_LS.bandNames()

# generate a list of images bases on a loop on band names. 
# Use band name to pick the image and then set a new property year generated from band name
list = bands.map(lambda b:  
    NDVI_LS.select([b]).
    rename(['NDVI_mean']).
    set({"year":ee.Number.parse(ee.String(b).split("_").get(1))})  
    )


# image collection from a list of images
NDVI_LS = ee.ImageCollection.fromImages(list)


# Visualize NDVI for 2000

In [4]:
myyear = 2000
myimage = NDVI_LS.filter(ee.Filter.eq("year", myyear)).first()

# Define visualization parameters
vis_params = {
    "bands": ["NDVI_mean"],  # Change bands if needed
    "min": -1,
    "max": 1, 
    "palette": ["red", "yellow", "green"]
}

# Create a map
Map = geemap.Map(center=[51.94, 16.35], zoom=5)  # Adjust center and zoom as needed

# Add the first image to the map
Map.addLayer(myimage, vis_params, f"LandSat 8 ({myyear})")
Map

Map(center=[51.94, 16.35], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

## Load percentiles per NUTS 


In [5]:
#use CSV saved in Google Drive URL to read percentiles table
PERCENTILES_TABLE_URL = f"{cfg_GDRIVE_BASE_URL}{cfg_GEE_PERCENTILES_FILE_ID}"

percentiles = pd.read_csv(PERCENTILES_TABLE_URL)

percentiles.to_csv(OUTPUT_DIR / 'tables' /  "percentiles.csv", index=False)

# dummy coordinates to user later for join
percentiles['longitude'] = 45
percentiles['latitude'] = 45


percentiles

,LEVL_CODE,NAME_LATN,NUTS_ID,NUTS_NAME,p2,p95,mean,longitude,latitude
0,2,Veri,AL01,Veri,0.259728,0.744132,0.567610,45,45
1,2,Qender,AL02,Qender,0.232497,0.732425,0.543386,45,45
2,2,Jug,AL03,Jug,0.244179,0.708973,0.512216,45,45
3,2,Burgenland,AT11,Burgenland,0.001650,0.755809,0.563257,45,45
4,2,Steiermark,AT22,Steiermark,0.380903,0.775324,0.680112,45,45
...,...,...,...,...,...,...,...,...,...
312,2,Devon,UKK4,Devon,0.439520,0.779232,0.688386,45,45
313,2,"Gloucestershire, Wiltshire and Bristol/Bath area",UKK1,"Gloucestershire, Wiltshire and Bristol/Bath area",0.384741,0.767529,0.635429,45,45
314,2,Dorset and Somerset,UKK2,Dorset and Somerset,0.400387,0.767510,0.661829,45,45
315,2,East Yorkshire and Northern Lincolnshire,UKE1,East Yorkshire and Northern Lincolnshire,0.263687,0.705075,0.528031,45,45


In [8]:
#Read NUTS
nuts = ee.FeatureCollection(cfg_nuts)

#Read FUA
fua = ee.FeatureCollection(cfg_FUA)

# get FUA bounding box
fua_bb = fua.geometry().bounds()

# Filter NUTS by FUA bounding box
nuts = nuts.filterBounds(fua_bb)

In [9]:
# get a python list with unique NUTS_IDs from the NUTS feature collection
NUTS_IDs= nuts.aggregate_array('NUTS_ID').getInfo() 


## Join NUTS fc with percentiles table

In [10]:
# convert percentiles DataFrame to an Earth Engine FeatureCollection
percentiles_ee = geemap.pandas_to_ee(percentiles)

# Create a filter to join NUTS and percentiles based on NUTS_ID
# Note: The leftField and rightField should match the field names in your FeatureCollection
# Ensure that the NUTS_ID field exists in both FeatureCollections
# If the field names are different, you can rename them in the FeatureCollection before applying the filter
NUTS_Filter = ee.Filter.equals(
  leftField = 'NUTS_ID',
  rightField='NUTS_ID'
)

# create an inner join between NUTS and percentiles_ee using the filter
innerJoin = ee.Join.inner()

# Apply the inner join to the NUTS and percentiles_ee FeatureCollections
nuts_Join = innerJoin.apply(nuts, percentiles_ee,  NUTS_Filter)

In [11]:
# Generate a list of years based on the start and end dates from the config
start_year = datetime.strptime(cfg_ls5_startDate, '%Y-%m-%d').year
end_year = datetime.strptime(cfg_ls8_endDate, '%Y-%m-%d').year

years = ee.List([f"NDVI_mean_{year}" for year in range(start_year, end_year) if year not in (2012, 2013)])

# Calculate FCOVER
Apply the **dichotomy** correction

In [12]:
# create a gee list of NUTS_IDs 
eeNUTS_IDs = ee.List(NUTS_IDs)

# Define a function to apply the formula to each image in the collection
def apply_correction(image, p2, p95):
    # Apply the formula to the image
    year = ee.Number(image.get('year')).format('%.0f')
    year = ee.String('green_').cat(year)

    
    # Second version F = (NDVI - NDVIo) / (NDVIoo - NDVIo)
    expr = ee.String("10000*((b(0)-").cat(p2).cat(")/").cat("(").cat(p95).cat("-").cat(p2).cat("))")
    

    result = image.expression(
        expr,  # formula to be applied
        #{'b': image}  # input variables
    ).rename(year)
    return result




# set values in range [0,1]
def set_values(image):
    expr = "b(0) > 10000 ? 10000 : (b(0) < 0 ? 0 : b(0))"
    img = image.expression(
        expr,
        {
            'b(0)': image.select(0)  
        }
    )
    return img.toUint16()

# Function to iterate over each feature and apply the mask to the ImageCollection
def apply_maskAndCorrection_to_collection(nutsid):
    
    feat = ee.FeatureCollection(nuts_Join.filter(ee.Filter.eq('secondary.NUTS_ID', ee.String(nutsid))))

    p2 = ee.Number(feat.first().get('secondary.p2')).format('%.3f')
    p95 = ee.Number(feat.first().get('secondary.p95')).format('%.3f')
    geom = ee.Geometry(ee.Feature(feat.first().get('primary')).geometry())
    

    # create a mask based on current feature
    mask = ee.Image.constant(1).clip(geom)
    
    masked_collection = NDVI_LS.map(lambda img: img.updateMask(mask))

    
    # apply correction        
    masked_collection = masked_collection.map(lambda img: apply_correction(img,p2=p2, p95=p95))
    
    masked_collection = masked_collection.map(lambda img: set_values(img))
    
    # convert imagecollection to multiband
    masked_collection =masked_collection.toBands() 
    
    # set NUTS_ID property
    masked_collection = masked_collection.set('NUTS_ID', nutsid)
    
    return  masked_collection.rename(years)

# for each ID of NUTS apply the function
# it returns a list with length equal to NUTS count and every element (NUTS) has an multiband with 19 corrected NDVIs (FCOVER)
nuts_images = eeNUTS_IDs.map(apply_maskAndCorrection_to_collection) # nuts_images

In [13]:
# get NUTS_IDs from nuts_images ee.List


# Function to extract the ID from each feature
def get_id(feature):
    return ee.Feature(feature).get('NUTS_ID')

# Map the function over the ee.List to get a new ee.List of IDs
id_list = nuts_images.map(get_id)

## Calculate FUA zonal statistics for FCOVER (mean and count) for each image of nuts_images ee.List

In [ ]:
# Define the reducers
mean_reducer = ee.Reducer.mean()
count_reducer = ee.Reducer.count()

# Combine the reducers
combined_reducer = mean_reducer.combine(count_reducer, sharedInputs=True)

def mean_zonal_statistics(image):
    image = ee.Image(image)
    NUTS_ID = image.get('NUTS_ID')
    # Apply reduceRegions to calculate the mean of all bands over the feature collection
    mean_stats = image.reduceRegions(
        collection=fua,
        reducer=combined_reducer,
        scale=100 ,
        crs="EPSG:3035"
    ).map(lambda feature: feature.setGeometry(None)).map(lambda feature: feature.set('NUTS_ID', NUTS_ID))
    
    return mean_stats

# Use map to apply the function over each image in the ee.List
mean_results = nuts_images.map(mean_zonal_statistics)

flattened_mean_results = ee.FeatureCollection(mean_results).flatten()



# task = ee.batch.Export.table.toDrive(
#     collection= flattened_mean_results,
#     folder = cfg_GDRIVE_EXPORT_DIRECTORY,
#     description='flattened_mean_results_EU',
#     fileFormat='CSV',
#     #selectors= ['LEVL_CODE', 'NAME_LATN', 'NUTS_ID', 'NUTS_NAME', 'p2','p95']
# )
# task.start()

In [15]:
# read the output CSV file from Google Drive
# first public share the CSV in gdrive and copy the ID

URL_ID = '1h5gSZvn1RPya19WoK9dCVEjfOZfSYhHm'
flattened_mean_results_EU_URL = f'{cfg_GDRIVE_BASE_URL}{URL_ID}'

gdf = pd.read_csv(flattened_mean_results_EU_URL)

# Find columns starting with 'NDVI_mean'
# keep mean remove count
ndvi_columns = gdf.filter(like='NDVI_mean').columns

# Remove rows with NaN in any of the NDVI_mean columns
gdf = gdf.dropna(subset=ndvi_columns)

# remove duplicates, keep only rows per fua_code with max count in one of the count columns, ie NDVI_mean_2020_count
idx = gdf.groupby('fua_code')['NDVI_mean_2020_count'].idxmax()
gdf = gdf.loc[idx].reset_index(drop=True)

In [16]:
# Convert FCOVER statistics to long format

# Drop columns that end with '_count'
gdf = gdf.drop(columns=gdf.filter(regex='_count$').columns)

# Pivot longer columns that start with 'NDVI_mean_'
gdf_long = pd.melt(
    gdf,
    id_vars=['fua_code', 'fua_name'],  # Columns to keep as identifiers
    value_vars=[col for col in gdf.columns if col.startswith('NDVI_mean_')],
    var_name='Year',
    value_name='FCOVER_mean'
)

# Extract only the numeric values from the 'Year' column
gdf_long['Year'] = gdf_long['Year'].str.extract('(\d+)')

# build output dir and create it if does not exist
output_dir = Path(OUTPUT_DIR) / "tables/FCOVER_mean"
output_dir.mkdir(parents=True, exist_ok=True)  

gdf_long['FCOVER_mean'] *= 100 # convert to percentage


gdf_long.to_csv( output_dir / f'{cfg_FCOVER_mean_FUA_CSV}_EU.csv', index=False)


In [15]:
gdf_long

,fua_code,fua_name,Year,FCOVER_mean
0,AL001L1,Tirana,2000,59.212968
1,AL003L1,Elbasan,2000,49.843413
2,AL004L1,ShkodÃ«r,2000,61.285749
3,AL005L0,VlorÃ«,2000,46.781498
4,AT001L3,Wien,2000,41.316002
...,...,...,...,...
13200,UK569L2,Ipswich,2020,44.946748
13201,UK571L1,Cheltenham,2020,67.759845
13202,XK001L1,Pristina,2020,66.614868
13203,XK002L1,Prizren,2020,69.771920


# Calculate slope of FCOVER per FUA (in degrees)

For use in web page and map

In [ ]:
from scipy.stats import linregress
import numpy as np

def get_slope_and_angle(group):
    group = group.sort_values('Year')
    
    # Ensure Year and FCOVER_mean are numeric
    group['Year'] = pd.to_numeric(group['Year'], errors='coerce')
    group['FCOVER_mean'] = pd.to_numeric(group['FCOVER_mean'], errors='coerce')

    group = group.dropna(subset=['Year', 'FCOVER_mean'])

    if len(group) < 2:
        return pd.Series({'slope': None, 'angle_degrees': None})

    slope, _, _, _, _ = linregress(group['Year'], group['FCOVER_mean'])

    # Convert slope to angle in degrees
    angle = np.degrees(np.arctan(slope))

    return pd.Series({
        'slope': round(slope, 2),
        'slope_degrees': round(angle, 2)
    })

# Apply groupby and get slope and angle per group
df_fcover_slope = gdf_long.groupby(['fua_code', 'fua_name']).apply(get_slope_and_angle).reset_index()
df_fcover_slope.to_csv( output_dir / f'FCOVER_slope.csv', index=False)

# Greenness calculation based on Kernel smoothing

In [18]:

# convert multiband to imagecollection
def bb(item):
    image = ee.Image(item)
    image_collection = ee.ImageCollection(image.bandNames().map(lambda band_name: image.select([band_name])))
    return(image_collection)

# a list of imagecollections
m = nuts_images.map(bb)

# Define the circular kernel with a radius of 5 pixels
kernel = ee.Kernel.circle(radius=5, units='pixels')


# Function to split the band name by "_" and get the last item, then append 'kernel'
def modify_band_name(band_name):
    # Split the band name by "_" and get the last element (the numeric part)
    numeric_part = ee.String(band_name).split('_').get(-1)  # Get the last item after the split
    
    new_name = ee.String('kernel_').cat(numeric_part)
    
    return  ee.String(new_name)

# for each item in list of imagecollections, reproject to epsg3035,100m and appy a kernel 5pixel . 
# Also set the NUTS_ID of each multiband image   
def applyKernel(item):
    item = ee.ImageCollection(item)
    NUTSID = item.first().get('NUTS_ID') # from the imagecollection(item) get first item and then its NUTS_ID property
    
    reprojected_collection = item.map(lambda img: img.reproject(crs="EPSG:3035", scale=100))

    convolved_collection = ee.ImageCollection(reprojected_collection.map(lambda img: img.convolve(kernel)))
    
    convolved_img= convolved_collection.toBands().set('NUTS_ID', NUTSID) # convert imagecollection to multiband image and assign NUTS_ID property
    
    # Rename the bands in the image with the new names
    # Get the band names
    band_names = convolved_img.bandNames()
    # Modify the band names by removing the first two characters
    new_band_names = band_names.map(modify_band_name)
    
    # Rename the bands in the image with the new names
    renamed_image = convolved_img.select(band_names).rename(new_band_names) 
    
    return(renamed_image)

smoothed = m.map(applyKernel)

### Calculate FUA zonal statistics for each Greenness images

In [19]:

# Define the reducers
mean_reducer = ee.Reducer.mean()
count_reducer = ee.Reducer.count()

# Combine the reducers
combined_reducer = mean_reducer.combine(count_reducer, sharedInputs=True)

def mean_zonal_statistics(image):
    image = ee.Image(image)
    NUTS_ID = image.get('NUTS_ID')
    # Apply reduceRegions to calculate the mean of all bands over the feature collection
    mean_stats = image.reduceRegions(
        collection=fua,
        reducer=combined_reducer,
        scale=100 ,
        crs="EPSG:3035"
    ).map(lambda feature: feature.setGeometry(None)).map(lambda feature: feature.set('NUTS_ID', NUTS_ID))
    
    return mean_stats

# Use map to apply the function over each image in the ee.List
mean_results = smoothed.map(mean_zonal_statistics)

flattened_mean_results = ee.FeatureCollection(mean_results).flatten()

task = ee.batch.Export.table.toDrive(
    collection= flattened_mean_results,
    folder = cfg_GDRIVE_EXPORT_DIRECTORY,
    description='Kernel_5px_mean_FCOVER_EU',
    fileFormat='CSV'
)
task.start()

# use the exported file from above command as input in smoothed_mean_results_EU_URL (the ID when the file is public shared) 


In [20]:
# wait for the previous task to finish, public share it and set the ID of the csv
CSV_ID = '1T-MC8_1VoAEFyTijVWrJ9-PDpgtaNcF8'
smoothed_mean_results_EU_URL = f'https://drive.google.com/uc?export=download&id={CSV_ID}'

gdf = pd.read_csv(smoothed_mean_results_EU_URL)

# Find columns contains _mean
ndvi_columns = gdf.filter(like='_mean').columns

# Remove rows with NaN in any of the NDVI_mean columns
gdf = gdf.dropna(subset=ndvi_columns)

# remove duplicates, keep only rows per fua_code with max count in one of the count columns, ie NDVI_mean_2020_count
idx = gdf.groupby('fua_code')['kernel_2020_count'].idxmax()
gdf = gdf.loc[idx].reset_index(drop=True)
gdf

,system:index,NUTS_ID,area,country,fid,fua_code,fua_name,kernel_2000_count,kernel_2000_mean,kernel_2001_count,...,kernel_2017_count,kernel_2017_mean,kernel_2018_count,kernel_2018_mean,kernel_2019_count,kernel_2019_mean,kernel_2020_count,kernel_2020_mean,perimeter,.geo
0,1_00000000000000000000,AL02,1.669981e+09,AL,1.0,AL001L1,Tirana,123504,0.585671,123562,...,123531,0.678225,123560,0.729395,123542,0.711499,123534,0.704748,309026.128094,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1_00000000000000000001,AL02,1.259253e+09,AL,2.0,AL003L1,Elbasan,126420,0.496867,126185,...,126403,0.644378,126449,0.716661,126382,0.696082,126308,0.684377,218480.332363,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,0_00000000000000000002,AL01,1.850448e+09,AL,3.0,AL004L1,ShkodÃ«r,175138,0.603258,176956,...,177115,0.661338,179504,0.708628,179149,0.713423,179605,0.638962,316821.461414,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,2_00000000000000000003,AL03,6.373832e+08,AL,4.0,AL005L0,VlorÃ«,61738,0.457126,61959,...,61872,0.580200,61790,0.624395,61711,0.613231,61695,0.625171,257243.965482,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,6_00000000000000000004,AT12,9.180279e+09,AT,5.0,AT001L3,Wien,740332,0.408590,741473,...,740651,0.403730,740923,0.456810,741402,0.435985,741040,0.506639,934507.239625,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,298_000000000000000002c1,UKH1,1.555006e+09,UK,784.0,UK569L2,Ipswich,127566,0.647248,153732,...,154344,0.618060,154336,0.564276,154287,0.662847,154335,0.444080,325972.634695,"{""type"":""MultiPoint"",""coordinates"":[]}"
691,313_000000000000000002c2,UKK1,4.618756e+08,UK,785.0,UK571L1,Cheltenham,46981,0.590530,47004,...,46998,0.743034,47012,0.675553,47005,0.732559,47011,0.670613,175788.663467,"{""type"":""MultiPoint"",""coordinates"":[]}"
692,245_000000000000000002c3,RS22,2.557848e+09,XK,786.0,XK001L1,Pristina,1180,0.517973,1180,...,1180,0.499542,1180,0.489254,1180,0.500844,1180,0.429410,373148.872950,"{""type"":""MultiPoint"",""coordinates"":[]}"
693,191_000000000000000002c4,MK00,1.275676e+09,XK,787.0,XK002L1,Prizren,212,0.240570,212,...,212,0.372103,212,0.395612,212,0.345299,212,0.376930,228187.199767,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [21]:
# Drop columns that end with '_count'
gdf = gdf.drop(columns=gdf.filter(regex='_count$').columns)

# Pivot longer columns that start with 'NDVI_max_'
gdf_long = pd.melt(
    gdf,
    id_vars=['fua_code', 'fua_name'],  # Columns to keep as identifiers
    value_vars=[col for col in gdf.columns if col.startswith('kernel_')],
    var_name='Year',
    value_name='kernel_mean'
)

# Extract only the numeric values from the 'Year' column
gdf_long['Year'] = gdf_long['Year'].str.extract('(\d+)')


#gdf_long.to_csv(output_dir / f'FCOVER_mean_FUA_EU_kernel-5px.csv', index=False)
gdf_long

,fua_code,fua_name,Year,kernel_mean
0,AL001L1,Tirana,2000,0.585671
1,AL003L1,Elbasan,2000,0.496867
2,AL004L1,ShkodÃ«r,2000,0.603258
3,AL005L0,VlorÃ«,2000,0.457126
4,AT001L3,Wien,2000,0.408590
...,...,...,...,...
13200,UK569L2,Ipswich,2020,0.444080
13201,UK571L1,Cheltenham,2020,0.670613
13202,XK001L1,Pristina,2020,0.429410
13203,XK002L1,Prizren,2020,0.376930


# Stack *Greenness* with *Population*

In [22]:
# WorldPop Global Project Population Data: Estimated Residential Population per 100x100m Grid Square

# Load the WorldPop ImageCollection specified in the configuration
PopGR = ee.ImageCollection(cfg_WorldPop)

# Filter the collection to include only images that intersect the bounding box of  FUA fc
PopGR = PopGR.filterBounds(fua_bb)

                    
# ---------------------------------------------
# Step 1: Extract all unique 'year' values from the filtered collection
# This will be used to create yearly mosaics
years = PopGR.aggregate_array('year').distinct()


# ---------------------------------------------
# Step 2: Define a function to create a mosaic for each year
# For each year:
#   - Filter the collection to get only images from that year
#   - Create a mosaic (combines overlapping images into one)
#   - Tag the mosaic with its corresponding year as metadata
def mosaic_by_year(y):
    y = ee.Number(y)
    filtered = PopGR.filter(ee.Filter.eq('year', y))
    mosaic = filtered.mosaic().set({
        'year': y
    })
    return mosaic

# ---------------------------------------------
# Step 3: Map the mosaic function over the list of unique years
# This produces a new ImageCollection with one mosaic image per year
mosaicked_PopGR = ee.ImageCollection(years.map(mosaic_by_year))

# ---------------------------------------------
# Step 4: Round population values and remove zeros

# Define a function to round the 'population' band values to the nearest integer
# This helps standardize values and ensures cleaner downstream analysis
def round_population(image):
    rounded = image.select('population').round()
    return image.addBands(rounded, overwrite=True)

# Apply the rounding function to each image in the collection
mosaicked_PopGR = mosaicked_PopGR.map(round_population)

# Mask out pixels with population less than 1 (i.e., 0 or negative values, if any)
# Also cast the image to 16-bit unsigned integer to optimize storage and compatibility
mosaicked_PopGR = mosaicked_PopGR.map(lambda image: image.updateMask(image.select("population").gte(1)).toUint16())

### Map Population for 2018

In [ ]:
myyear = 2018
myimage = mosaicked_PopGR.filter(ee.Filter.eq("year", myyear)).first()

# Define visualization parameters
vis_params = {
    "bands": ["population"],  # Change bands if needed
    "min": 1,
    "max": 70, 
    "palette": ["red", "yellow", "green"]
}

# Create a map
Map = geemap.Map(center=[51.94, 16.35], zoom=5)  # Adjust center and zoom as needed

# Add the first image to the map
Map.addLayer(myimage, vis_params, f"LandSat 8 ({myyear})")
Map

### Convert a list of multiband Greenness images (*smoothed*) to a list of Greenness image collections (*smoothed_list_col*)


In [23]:
def tocol(item):
    # get band names
    bands = ee.Image(item).bandNames()

    # generate a list of images bases on a loop on band names. 
    # Use band name to pick the image and then set a new property year generated from band name
    list = bands.map(lambda b:  
         ee.Image(item).select([b]).
        set({"year":ee.Number.parse(ee.String(b).split("_").get(1))})  
        )

    # image collection from a list of images
    LS = ee.ImageCollection.fromImages(list)
    return(LS)
    
smoothed_list_col = smoothed.map(tocol) #  a list of imagecollections (Greenness)

### Convert a mosaicked greenness imagecollection, one full extent image per year 

In [24]:
# Convert a list of image collections (smoothed_list_col) to a single image collection (flattened)
# So it will end up in multiple images per year covering different NUTS
# Then, at Step 3, we will mosaic the images per year

# Step 1: Flatten all collections into one big collection
flattened = ee.ImageCollection(smoothed_list_col.iterate(
    lambda col, acc: ee.ImageCollection(acc).merge(ee.ImageCollection(col)),
    ee.ImageCollection([])
))



# Step 2: Get unique years from 'year' property
years = flattened.aggregate_array('year').distinct()

# Step 3: Mosaic images per year
def mosaic_by_year(y):
    year = ee.Number(y)
    filtered = flattened.filter(ee.Filter.eq('year', year))
    mosaic = filtered.mosaic().set({
        'year': year

    })
    return mosaic

# Step 4: Map over years to build a new ImageCollection for Greenness
mosaicked_collection = ee.ImageCollection(years.map(mosaic_by_year)) 

### Reproject and rescale greenness mosaicked_collection based on PopGR properties

In [25]:

# Step 1: Get projection from first image in PopGR
reference_image = ee.Image(PopGR.first())
reference_proj = reference_image.select(0).projection()  # use first band as reference

# Step 2: Reproject each image in greenness image  to match CRS and scale of PopGR
def reproject_image(image):
    return ee.Image(image).reproject(crs=reference_proj.crs(), scale=reference_proj.nominalScale())

# Step 3: Apply reprojection to imgB
mosaicked_collection_reproj = mosaicked_collection.map(reproject_image)


# stack Population and Vegetation

In [26]:

# stack Population and Vegetation
def stack(img):
    myyear = img.get('year')
 
    img = ee.Image(img.select([0]).rename('fcover'))
   
    
    # Select population image for current year
    PopGR_yr = ee.Image(mosaicked_PopGR.filter(ee.Filter.eq('year', myyear)).first() \
        .select('population').toUint16())
    
    # Apply common mask
    common_mask = img.mask().And(PopGR_yr.mask())
    img_masked = img.updateMask(common_mask)
    pop_masked = PopGR_yr.updateMask(common_mask)
    
    # Merge bands with common mask
    merged_img = img_masked.addBands(pop_masked, None, True).toUint16()

    return merged_img
 
# apply stack function on imagecollection
PopVeg = mosaicked_collection_reproj.map(stack)

# Convert all bands in every image to int
#PopVeg = PopVeg.map(lambda img: img.toUint16())


# ✅ Now stacked_collection contains images from imgA + corresponding bands from imgB
print("Stacked ImageCollection created.")

Stacked ImageCollection created.


# Export *PopVeg* Stacked image collection as as asset (or gdrive) images 

In [ ]:
#fua_sel = fua.filter(ee.Filter.eq('fua_code', 'RO019L1'))

export_as_asset = True
export_inGdrive = False

nodata_value = np.iinfo(np.uint16).max

Years = [year for year in range(start_year, end_year) if year not in (2012, 2013)]

for Year in Years:  
    
    img = PopVeg.filter(ee.Filter.eq('year', Year)).first() 
        
    
    
    geom = fua.geometry().transform('EPSG:3035', 100)
    
    #Export region (should be a geometry or bounding box)
    region = geom.bounds(1).getInfo()['coordinates']
    
   
    # Create a mask where pixels inside the feature are 1, else 0
    mask = ee.Image.constant(1).clip(geom).mask()

    
    masked_img = img.updateMask(mask).unmask(nodata_value)
    
    assetId = f'asset_PopVeg_Image_{Year}_Uint16_nozeros'
    

    

    # --- Export the masked image to an Earth Engine asset
    if export_as_asset:        
        geemap.ee_export_image_to_asset(
            masked_img, 
            description=assetId, 
            assetId=assetId, 
            region=region, #fua.geometry(),
            scale=100, 
            crs="EPSG:3035",
            maxPixels=1e13)
        
        
        
    
    # --- Export the masked image to Google Drive    
    if export_inGdrive:        
        geemap.ee_export_image_to_drive(
            image=masked_img,
            description=assetId,      # Use the same ID as the task description
            folder='asset_PopVeg_Images_Uint16',            # Destination folder in your Google Drive
            fileNamePrefix=assetId,   # Filename of the exported image
            region=region,            # Export region (should be a geometry or bounding box)
            scale=100,                # Resolution in meters
            crs="EPSG:3035",          # Coordinate reference system
            maxPixels=1e13,            # Increase if exporting large images
            fileFormat='GeoTIFF', 
            formatOptions={
                'noData': np.iinfo(np.uint16).max
            }
    )

    print(f"Export {Year} started. Check Earth Engine Tasks tab or your Google Drive.")

Export 2001 started. Check Earth Engine Tasks tab or your Google Drive.
